In [1]:
%pip install langchain tiktoken chromadb langchain-nvidia-ai-endpoints -U -q

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\anubh\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\onnxruntime\\tools\\ort_format_model\\ort_flatbuffers_py\\fbs\\Dimension.py'
Check the permissions.


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
%pip install langchain_community openai -U -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\anubh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
%pip show chromadb

Name: chromadb
Version: 1.0.15
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: C:\Users\anubh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: bcrypt, build, grpcio, httpx, importlib-resources, jsonschema, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-sdk, orjson, overrides, posthog, pybase64, pydantic, pypika, pyyaml, rich, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
!curl -L -o new_articles.zip https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   124  100   124    0     0    120      0  0:00:01  0:00:01 --:--:--   120

100    17  100    17    0     0     10      0  0:00:01  0:00:01 --:--:--    10
100    17  100    17    0     0     10      0  0:00:01  0:00:01 --:--:--     0

  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100   470    0   470    0     0    146      0 --:--:--  0:00:03 --:--:--  2919

  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
100 79974  100 79974    0     0  20743      0  0:00:03  0:00:03 --:--:--  373k


In [5]:
import zipfile
with zipfile.ZipFile("new_articles.zip", "r") as zip_ref:
    zip_ref.extractall("new_articles")

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
NVIDIA_KEY = os.getenv("NVIDIA_API_KEY")
OPENROUTER_KEY = os.getenv("OPENROUTER_KEY")

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.document_loaders import TextLoader , DirectoryLoader
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from openai import OpenAI


In [4]:
def OpenRouterChat(query):
    client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_KEY,
    )
    completion = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
        {
        "role": "user",
        "content": query
        }
    ]
    )
    return completion.choices[0].message.content

In [5]:
def NVIDIAChat(query):
    client = ChatNVIDIA(
    model="deepseek-ai/deepseek-r1",
    api_key=NVIDIA_KEY, 
    temperature=0.6,
    top_p=0.7,
    max_tokens=4096,
    )
    response = client.invoke(
        [
            {
                "role": "user",
                "content": query
            }
        ]
    )
    return response.content

In [6]:
print(OpenRouterChat("What are the precautions we should take on a rainy day?"))

On rainy days, taking precautions can ensure safety, health, and comfort. Here’s a structured guide:

### **1. Clothing & Personal Safety**
- **Waterproof Gear:** Wear a raincoat, waterproof jacket, or carry a sturdy umbrella. Opt for quick-dry fabrics.
- **Footwear:** Use anti-slip, closed-toe shoes or boots to avoid slips and fungal infections.
- **Visibility:** Wear bright or reflective clothing if walking/biking to stay visible to drivers.

### **2. Travel Precautions**
- **Driving:** 
  - Reduce speed, maintain a safe distance, and avoid sudden braking.
  - Ensure windshield wipers, headlights, and tires are functional.
  - Avoid flooded roads; even shallow water can stall engines.
- **Walking:** 
  - Steer clear of puddles (hidden hazards like open drains or debris).
  - Avoid isolated trees, poles, or downed power lines (lightning risk).

### **3. Home Safety**
- **Drainage:** Clean gutters and drains to prevent waterlogging.
- **Flood Prevention:** Use sandbags if prone to floo

In [7]:
# print(NVIDIAChat("What are the precautions we should take on a rainy day?"))
"""
Apparently NVIDIA API is not working at the moment, so I am using OpenRouter API.
"""

'\nApparently NVIDIA API is not working at the moment, so I am using OpenRouter API.\n'

In [10]:
loader = DirectoryLoader(
    "new_articles",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
    show_progress=True
)
#loading all the news articles from the directory

In [11]:
document = loader.load()

100%|██████████| 21/21 [00:00<00:00, 60.89it/s]


In [12]:
#splitting into text chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
text = text_splitter.split_documents(document)

# Creating DB

In [ ]:
# Creating DB using NVIDIA embeddings and Chroma vector store
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedding = NVIDIAEmbeddings(
  model="NV-Embed-QA", 
  api_key=NVIDIA_KEY
  )

# embeddings = embedding.embed_query("hello world")

from langchain_community.vectorstores import Chroma


vectorstore = Chroma.from_documents(
    documents=text,
    embedding=embedding,
    persist_directory="./nvidia_chroma_db"
)

vectorstore.persist()

C:\Users\anubh\AppData\Local\Temp\ipykernel_31368\95557075.py:20: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


# Retriever

In [20]:
retriever = vectorstore.as_retriever()
# Retriever function to query the vector database
# This function takes a query string and returns relevant documents from the vector database
def query_vector_db(query):
    results = retriever.get_relevant_documents(query)
    return results

# Example query
query = "How AI will be the next Apex Intelligence?"
results = query_vector_db(query)

In [30]:
print(results[0].page_content)

Slack has evolved from a pure communications platform to one that enables companies to link directly to enterprise applications without having to resort to dreaded task switching. Today, at the Salesforce World Tour event in NYC, the company announced the next step in its platform’s evolution where it will be putting AI at the forefront of the user experience, making it easier to get information and build workflows.

It’s important to note that these are announcements, and many of these features are not available yet.

Rob Seaman says that rather than slapping on an AI cover, they are working to incorporate it in a variety of ways across the platform. That started last month with a small step, a partnership with OpenAI to bring a ChatGPT app into Slack, the first piece of a much broader vision for AI on the platform. That part is in beta at the moment.


In [32]:
query = "What the headlines at Wall Street Journal?"

In [33]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
results = retriever.get_relevant_documents(query)

In [34]:
print(results[0].page_content)

Apple releases security fix: On Monday, Apple released its first batch of publicly available “rapid security” patches, aimed at quickly fixing security vulnerabilities that are under active exploitation or pose significant risks to its customers. Apple says that these patches, which are enabled by default, were intended to let customers update their devices faster than a typical software upgrade.

Musk settles for less: A defamation case brought against Tesla chief executive Elon Musk by critic Randeep Hothi is coming to a close, reportedly costing the billionaire ten big ones. Lawyers representing Hothi — a vocal member of the TSLAQ short-seller community on Twitter who rose to prominence as a skeptic of Tesla’s gigafactory plans and “full self-driving” tech — said in a statement that Musk asked to settle the nearly three-year-old case back in March.


A RetrievalQA chain connects:

User query → Retriever (search relevant docs) → LLM (generate final answer)



In [43]:
from langchain_core.outputs import LLMResult, Generation
from langchain_core.language_models import BaseLLM
from openai import OpenAI

class OpenRouterLLM(BaseLLM):
    model: str
    api_key: str

    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop=None):
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.api_key
        )
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ]
        )
        return response.choices[0].message.content

    def _generate(self, prompts, stop=None, **kwargs):
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop=stop)
            generations.append([Generation(text=text)])
        return LLMResult(generations=generations)

In [44]:
#initializing the OpenRouterLLM
llm = OpenRouterLLM(
    model="deepseek/deepseek-r1:free",
    api_key=OPENROUTER_KEY
)
retriever = vectorstore.as_retriever()

In [45]:
#setting up the RetrievalQA chain
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)


In [ ]:
query = "How much money did Microsoft raise?"
response = qa_chain.invoke(query)

print("📌 Answer:\n", response["result"])

print("\n📚 Sources:")
for doc in response["source_documents"]:
    print("-", doc.metadata.get("source", "Unknown"))


📌 Answer:
 Microsoft did not raise money; instead, Microsoft **invested** in OpenAI. According to the context, Microsoft's investment in OpenAI closed in January 2023 and was around **$10 billion**, separate from the VC investments totaling over $300 million made in April 2023. 

**Answer:** Microsoft invested approximately $10 billion in OpenAI.

📚 Sources:
- new_articles\05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
- new_articles\05-06-this-week-in-apps-apple-and-google-team-up-on-trackers-google-i-o-preview-apps-hit-newfronts.txt
- new_articles\05-07-fintech-space-continues-to-be-competitive-and-drama-filled.txt
- new_articles\05-03-spawning-lays-out-its-plans-for-letting-creators-opt-out-of-generative-ai-training.txt


: 